# Water Jug Problem

This notebook shows how to set up and solve a Water Jug Problem as an Integer Linear Program (ILP)

## What is the Water Jug Problem?

Suppose you have the following:
* An empty 5 gallon jug (Jug A)
* An empty 3 gallon jug (Jug B)
* An "infinite" water source like a spring

Pour water into the jugs such that you end up with:
* The 5 gallon jug containing 4 gallons
* The 3 gallon jug containing 3 gallons

You're allowed to make the following moves:
* Dump out all of the water Jug A
* Dump out all of the water Jug B
* Completely fill up Jug A
* Completely fill up Jug B
* Pout water from one jug into the other

Which moves (pours/dumps/fills) should you make such that the total number of moves is minimized?

# Imports

In [1]:
from docplex.mp.model import Model

# Sets

* $T$: The set of time periods
* $J$: The set of water jugs
* $D$: List of possible moves

In [2]:
T = list(range(11))
J = ['A','B']
D = list(range(6))

moves = {
    0: 'Dump A',
    1: 'Dump B',
    2: 'Fill A',
    3: 'Fill B',
    4: 'Pour A to B',
    5: 'Pour B to A'
}

# Parameters

* $c_{j}$: Capcity for jug j
* $f_{j}$: Final volume for jug j


In [3]:
c = {
    'A':5,
    'B':3
}

f = {
    'A':4,
    'B':3
}

# Create Optimization Model

In [4]:
# create a model object
water_jug = Model(name='Water-Jug-Problem')

# Decision Variables

* $z_{dt}$: A binary variable that equals 1 if we make decision $d$ at time $t$ and 0 otherwise

In [5]:
z = water_jug.binary_var_dict(keys = [(d, t) for d in D for t in T[1:]], name="z")

* $v_{jt}$: The volume of water in jug $j$ at time $t$

In [6]:
def get_ub(var_name):
    return c[var_name[0]]

v = water_jug.integer_var_dict(keys = [(j, t) for j in J for t in T], name="v",lb=0, ub=get_ub)

# Objective Function

Minimize the total number of moves (pours/fills/dumps) made

minimize $\sum_{d \in D, t \in T>0}z_{dt}$

In [7]:
num_moves = water_jug.sum(z[(d,t)] for d in D for t in T[1:])
water_jug.minimize(num_moves)

# Constraints

Both jugs are empty at the start

$v_{j0} = 0, \quad \forall j \in J$

In [8]:
water_jug.add_constraints(v[j,0] == 0 for j in J)

[docplex.mp.LinearConstraint[](v_A_0,EQ,0),
 docplex.mp.LinearConstraint[](v_B_0,EQ,0)]

At most, a single move can be made during each time period

$\sum_{d \in D}z_{dt} <= 1, \quad \forall t \in T>0$

In [9]:
water_jug.add_constraints(water_jug.sum(z[(d,t)] for d in D) <= 1 for t in T[1:])

[docplex.mp.LinearConstraint[](z_0_1+z_1_1+z_2_1+z_3_1+z_4_1+z_5_1,LE,1),
 docplex.mp.LinearConstraint[](z_0_2+z_1_2+z_2_2+z_3_2+z_4_2+z_5_2,LE,1),
 docplex.mp.LinearConstraint[](z_0_3+z_1_3+z_2_3+z_3_3+z_4_3+z_5_3,LE,1),
 docplex.mp.LinearConstraint[](z_0_4+z_1_4+z_2_4+z_3_4+z_4_4+z_5_4,LE,1),
 docplex.mp.LinearConstraint[](z_0_5+z_1_5+z_2_5+z_3_5+z_4_5+z_5_5,LE,1),
 docplex.mp.LinearConstraint[](z_0_6+z_1_6+z_2_6+z_3_6+z_4_6+z_5_6,LE,1),
 docplex.mp.LinearConstraint[](z_0_7+z_1_7+z_2_7+z_3_7+z_4_7+z_5_7,LE,1),
 docplex.mp.LinearConstraint[](z_0_8+z_1_8+z_2_8+z_3_8+z_4_8+z_5_8,LE,1),
 docplex.mp.LinearConstraint[](z_0_9+z_1_9+z_2_9+z_3_9+z_4_9+z_5_9,LE,1),
 docplex.mp.LinearConstraint[](z_0_10+z_1_10+z_2_10+z_3_10+z_4_10+z_5_10,LE,1)]

Note: These are indicator constraints. By using `add_indicator`, you can let CPLEX linearize these constraints.

* If you dump water out of a jug, then the corresponding jug will become empty

    * if $z_{0t} = 1$, then $v_{At} = 0, \quad \forall t \in T > 0$
    * if $z_{1t} = 1$, then $v_{Bt} = 0, \quad \forall t \in T > 0$

* If you fill a jug with water, then it will become full

    * if $z_{2t} = 1$, then $v_{At} = c_{A}, \quad \forall t \in T > 0$
    * if $z_{3t} = 1$, then $v_{Bt} = c_{B}, \quad \forall t \in T > 0$

In [10]:
# add dumping/filling constraints
for t in T[1:]:

    # dump A
    water_jug.add_indicator(z[(0,t)], v[('A',t)]==0)

    # dump B
    water_jug.add_indicator(z[(1,t)], v[('B',t)]==0)

    # Fill A
    water_jug.add_indicator(z[(2,t)], v[('A',t)]==c['A'])

    # Fill B
    water_jug.add_indicator(z[(3,t)], v[('B',t)]==c['B'])

Note: By using `add_if_then`, CPLEX will linearize the constraint for you

If you don't dump, fill, or pour into/out of jug A, then jug A has the same volume as before

if $z_{0t} + z_{2t} + z_{4t} + z_{5t} = 0$, then $v_{At} = v_{At-1}, \quad \forall t \in T>0$

In [11]:
for t in T[1:]:
    water_jug.add_if_then(z[(0,t)] + z[(2,t)] + z[(4,t)] + z[(5,t)] == 0, v[('A',t)] == v[('A',t-1)])


If you don't dump, fill, or pour into/out of jug B, then jug B has the same volume as before

if $z_{1t} + z_{3t} + z_{4t} + z_{5t} = 0$, then $v_{Bt} = v_{Bt-1}, \quad \forall t \in T>0$

In [12]:
for t in T[1:]:
    water_jug.add_if_then(z[(1,t)] + z[(3,t)] + z[(4,t)] + z[(5,t)] == 0, v[('B',t)] == v[('B',t-1)])

Note: CPLEX will linearize min and max constraints

if you pour A into B:

* if $z_{4t} = 1$, then $v_{At} = max(0, v_{At-1}+v_{Bt-1}-3), \quad \forall t \in T>0$ 
* if $z_{4t} = 1$, then $v_{Bt} = min(3, v_{At-1}+v_{Bt-1}), \quad \forall t \in T>0$ 

In [13]:
for t in T[1:]:
    
    # for A
    water_jug.add_indicator(z[(4,t)], v[('A',t)]==water_jug.max(0, v[('A',t-1)]-3+v[('B',t-1)]  ))

    # for B
    water_jug.add_indicator(z[(4,t)], v[('B',t)]==water_jug.min(3, v[('A',t-1)]+ v[('B',t-1)]  ))

if you pour B into A:

* if $z_{5t} = 1$, then $v_{At} = min(5, v_{At-1}+v_{Bt-1}), \quad \forall t \in T>0$ 
* if $z_{5t} = 1$, then $v_{Bt} = max(0, v_{At-1}+v_{Bt-1}-5), \quad \forall t \in T>0$ 

In [14]:
for t in T[1:]:
    
    # for A
    water_jug.add_indicator(z[(5,t)], v[('A',t)]==water_jug.min(5, v[('A',t-1)]+ v[('B',t-1)]  ))

    # for B
    water_jug.add_indicator(z[(5,t)], v[('B',t)]==water_jug.max(0, v[('A',t-1)]+ v[('B',t-1)] -5 ))

You can't make a decision in the current time period if you didn't make one in the previous time period

$ \sum_{d \in D}z_{dt+1} \le \sum_{d \in D}z_{dt}, \quad \forall t \in 0 < T < |T|$  

In [15]:
water_jug.add_constraints(water_jug.sum(z[(d,t+1)] for d in D) <= water_jug.sum(z[(d,t)] for d in D) for t in T[1:-1])

[docplex.mp.LinearConstraint[](z_0_2+z_1_2+z_2_2+z_3_2+z_4_2+z_5_2,LE,z_0_1+z_1_1+z_2_1+z_3_1+z_4_1+z_5_1),
 docplex.mp.LinearConstraint[](z_0_3+z_1_3+z_2_3+z_3_3+z_4_3+z_5_3,LE,z_0_2+z_1_2+z_2_2+z_3_2+z_4_2+z_5_2),
 docplex.mp.LinearConstraint[](z_0_4+z_1_4+z_2_4+z_3_4+z_4_4+z_5_4,LE,z_0_3+z_1_3+z_2_3+z_3_3+z_4_3+z_5_3),
 docplex.mp.LinearConstraint[](z_0_5+z_1_5+z_2_5+z_3_5+z_4_5+z_5_5,LE,z_0_4+z_1_4+z_2_4+z_3_4+z_4_4+z_5_4),
 docplex.mp.LinearConstraint[](z_0_6+z_1_6+z_2_6+z_3_6+z_4_6+z_5_6,LE,z_0_5+z_1_5+z_2_5+z_3_5+z_4_5+z_5_5),
 docplex.mp.LinearConstraint[](z_0_7+z_1_7+z_2_7+z_3_7+z_4_7+z_5_7,LE,z_0_6+z_1_6+z_2_6+z_3_6+z_4_6+z_5_6),
 docplex.mp.LinearConstraint[](z_0_8+z_1_8+z_2_8+z_3_8+z_4_8+z_5_8,LE,z_0_7+z_1_7+z_2_7+z_3_7+z_4_7+z_5_7),
 docplex.mp.LinearConstraint[](z_0_9+z_1_9+z_2_9+z_3_9+z_4_9+z_5_9,LE,z_0_8+z_1_8+z_2_8+z_3_8+z_4_8+z_5_8),
 docplex.mp.LinearConstraint[](z_0_10+z_1_10+z_2_10+z_3_10+z_4_10+z_5_10,LE,z_0_9+z_1_9+z_2_9+z_3_9+z_4_9+z_5_9)]

By the final time period, each jug needs to have its required volume of water

$v_{j|T|} = f_{j}, \quad \forall j \in J$

In [16]:
water_jug.add_constraint(v['A',max(T)] == f['A'])
water_jug.add_constraint(v['B',max(T)] == f['B'])

docplex.mp.LinearConstraint[](v_B_10,EQ,3)

# Solve the Problem

In [17]:
water_jug.solve()

docplex.mp.solution.SolveSolution(obj=6,values={z_1_3:1,z_2_1:1,z_2_5:1,..

# Output the solution

## Objective Function: Number of moves made

In [18]:
water_jug.solution.objective_value

6.0

## Moves that were made

In [19]:
z_values = water_jug.solution.get_value_df(z)
z_values = z_values.loc[z_values.value == 1].sort_values(by=['key_2'])
z_values["Move"] = z_values["key_1"].apply(lambda x: moves[x])
z_values.to_csv("sol-moves.csv",index=False)

In [20]:
z_values

,key_1,key_2,value,Move
20,2,1,1.0,Fill A
41,4,2,1.0,Pour A to B
12,1,3,1.0,Dump B
43,4,4,1.0,Pour A to B
24,2,5,1.0,Fill A
45,4,6,1.0,Pour A to B


## Volume at each time period

In [21]:
v_values = water_jug.solution.get_value_df(v)
v_values.to_csv("sol-volume.csv",index=False)

In [22]:
v_values

,key_1,key_2,value
0,A,0,0.0
1,A,1,5.0
2,A,2,2.0
3,A,3,2.0
4,A,4,0.0
5,A,5,5.0
6,A,6,4.0
7,A,7,4.0
8,A,8,4.0
9,A,9,4.0
